In [16]:
import json
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import numpy as np

In [2]:
stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

Downloading functionality is broken... get data from 
<a href="http://nltk.org/nltk_data/">NLTK Downloads</a>.

In [2]:
# nltk.download('twitter_samples')
# nltk.download('stopwords')

In [3]:
def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)
    return tweets_clean

In [4]:
def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
            #freqs.get(pair, 0) += 1
    return freqs

In [15]:
!ls ../../../../data/twitter_samples

README.md                   positive_tweets.json
README.txt                  tweets.20150430-223406.json
negative_tweets.json


In [23]:
DATA = '../../../../data'

In [24]:
def get_all_tweets(pos_neg):
    path = f'{DATA}/twitter_samples/{pos_neg}_tweets.json'
    all_tweets = []
    with open(path) as f:
        for line in f:
            tweet = json.loads(line)
            all_tweets.append(tweet['text'])
    return all_tweets

In [25]:
all_positive_tweets = get_all_tweets('positive')
all_negative_tweets = get_all_tweets('negative')
all_positive_tweets[:5]

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days']

In [26]:
# split the data into two pieces, one for training and one for testing 
# (validation set) 
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]
train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [27]:
# combine positive and negative labels
train_y = np.append(
    np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(
    np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [29]:
# Print the shape train and test sets
print('train_y.shape:', train_y.shape)
print('test_y.shape', test_y.shape)

train_y.shape: (8000, 1)
test_y.shape (2000, 1)


In [30]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

In [33]:
# check the output
print('type(freqs):', type(freqs))
print('len(freqs):', len(freqs))

type(freqs): <class 'dict'>
len(freqs): 11380


In [61]:
list(freqs.keys())[:5]

[('followfriday', 1.0),
 ('top', 1.0),
 ('engag', 1.0),
 ('member', 1.0),
 ('commun', 1.0)]

In [34]:
# test the function below
print('This is an example of a positive tweet:\n', train_x[0])
print('\nThis is an example of the processed version of the tweet:\n', 
      process_tweet(train_x[0]))

This is an example of a positive tweet:
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet:
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [35]:
def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        logit: the sigmoid of z
    '''
    logit = 1 / (1 + np.exp(-z))
    return logit

In [36]:
print(sigmoid(0))   # 0.5
print(sigmoid(5))   # near 1
print(sigmoid(12))  # very near 1
print(sigmoid(-5))  # near 0
print(sigmoid(-12)) # very near 0

0.5
0.9933071490757153
0.9999938558253978
0.0066928509242848554
6.144174602214718e-06


In [54]:
def get_loss(m, logit, y):
    '''Binary cross-entropy'''
    loss = (np.dot(y.T, np.log(logit)) 
            + np.dot((1 - y).T, np.log(1 - logit)))
    loss /= -m
    return loss

In [55]:
print(get_loss(1, np.array(sigmoid(7)), np.array([1])))
print(get_loss(1, np.array(sigmoid(7)), np.array([-1])))
print(get_loss(1, np.array(sigmoid(0)), np.array([1])))
print(get_loss(2, np.array([sigmoid(7), sigmoid(-12)]), np.array([1, 0])))

[0.00091147]
[14.00091147]
[0.69314718]
0.00045880532362596376


In [56]:
def gradientDescent(x, y, theta, alpha, n_iters, verbose=False):
    '''
    Input:
        x: matrix of features which is (m, n+1)
        y: corresponding labels of the input matrix x, dimensions (m, 1)
        theta: weight vector of dimension (n+1, 1)
        alpha: learning rate
        n_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is 
      going down.
    '''
    m = x.shape[0]    
    for i in range(n_iters):
        # get z, the dot product of x and theta
        z = np.dot(x, theta)
        # get the sigmoid of z
        logit = sigmoid(z)
        # calculate the cost function
        J = get_loss(m, logit, y)
        # update the weights theta
        delta = (alpha/m) * np.dot(x.T, logit - y)
        theta -= delta
        if verbose:
            print('Loss:', J)
    J = float(J)
    return J, theta

In [58]:
np.random.seed(1)
# X input is 10 x 3 with ones for the bias terms
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Y Labels are 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Apply gradient descent
tmp_J, tmp_theta = gradientDescent(
    tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is "
      f"{[round(t, 8) for t in np.squeeze(tmp_theta)]}")

The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]


In [60]:
process_tweet(train_x[0])

['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']

In [68]:
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple
        (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3))
    #bias term is set to 1
    x[0, 0] = 1 
    # loop through each word in the list of words
    for word in word_l:       
        # increment the word count for the positive label 1
        x[0, 1] += freqs.get((word, 1.), 0)
        # increment the word count for the negative label 0
        x[0, 2] += freqs.get((word, 0.), 0)
    assert(x.shape == (1, 3))
    return x

In [70]:
# Test
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1.00e+00 3.02e+03 6.10e+01]]
[[1. 0. 0.]]


In [71]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f'The cost after training is {J:.8f}.')
print(f'The resulting vector of weights is '
      f'{[round(t, 8) for t in np.squeeze(theta)]}')

The cost after training is 0.24213043.
The resulting vector of weights is [7e-08, 0.00052426, -0.00055472]


In [72]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple
          (word, label)
        theta: (3, 1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)
    # make the prediction using x and theta
    y_pred = sigmoid(np.dot(x, theta))
    return y_pred

In [78]:
# Run this cell to test your function
for tweet in [
    'I am happy', 'I am bad', 'this movie should have been great.', 
    'great', 'great great', 'great great great', 
    'great great great great', ':) :) :)', ':) :(', ':( :( :(']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

I am happy -> 0.518597
I am bad -> 0.494346
this movie should have been great. -> 0.515348
great -> 0.515478
great great -> 0.530925
great great great -> 0.546314
great great great great -> 0.561615
:) :) :) -> 0.988731
:) :( -> 0.368194
:( :( :( -> 0.002251


In [79]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    # the list for storing predictions
    y_hat = []    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1.)
        else:
            # append 0 to the list
            y_hat.append(0.)
    # With the above implementation, y_hat is a list, but test_y is (m, 1)
    # array convert both to one-dimensional arrays in order to compare 
    # them using the '==' operator
    y_hat = np.array(y_hat).reshape(-1, 1)
    accuracy = (y_hat == test_y).sum() / len(test_y)
    return accuracy

In [80]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9945


In [85]:
print('Label Predicted Tweet')
for x, y in zip(test_x, test_y):
    y_hat = predict_tweet(x, freqs, theta)
    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE TWEET IS:', x)
        print('THE PROCESSED TWEET IS:', process_tweet(x))
        print('%d\t%0.8f\t%s' 
              % (y, 
                 y_hat, 
                 ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

Label Predicted Tweet
THE TWEET IS: @jaredNOTsubway @iluvmariah @Bravotv Then that truly is a LATERAL move! Now, we all know the Queen Bee is UPWARD BOUND : ) #MovingOnUp
THE PROCESSED TWEET IS: ['truli', 'later', 'move', 'know', 'queen', 'bee', 'upward', 'bound', 'movingonup']
1	0.49999751	b'truli later move know queen bee upward bound movingonup'
THE TWEET IS: @MarkBreech Not sure it would be good thing 4 my bottom daring 2 say 2 Miss B but Im gonna be so stubborn on mouth soaping ! #NotHavingit :p
THE PROCESSED TWEET IS: ['sure', 'would', 'good', 'thing', '4', 'bottom', 'dare', '2', 'say', '2', 'miss', 'b', 'im', 'gonna', 'stubborn', 'mouth', 'soap', 'nothavingit', ':p']
1	0.48664967	b'sure would good thing 4 bottom dare 2 say 2 miss b im gonna stubborn mouth soap nothavingit :p'
THE TWEET IS: I'm playing Brain Dots : ) #BrainDots
http://t.co/UGQzOx0huu
THE PROCESSED TWEET IS: ["i'm", 'play', 'brain', 'dot', 'braindot']
1	0.48375947	b"i'm play brain dot braindot"
THE TWEET IS: I'm p

In [86]:
# Feel free to change the tweet below
my_tweet = (
    'This is a ridiculously bright movie :). The plot was terrible and I '
    'was sad until the ending!')
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['ridicul', 'bright', 'movi', ':)', 'plot', 'terribl', 'wassad', 'end']
[[0.81304644]]
Positive sentiment
